# License

Copyright Snap Inc. 2021. This sample code is made available by Snap Inc. for informational purposes only. 
No license, whether implied or otherwise, is granted in or to such code (including any rights to copy, modify, 
publish, distribute and/or commercialize such code), unless you have entered into a separate agreement for such rights. 
Such code is provided as-is, without warranty of any kind, express or implied, including any warranties of merchantability, 
title, fitness for a particular purpose, non-infringement, or that such code is free of defects, errors or viruses. 
In no event will Snap Inc. be liable for any damages or losses of any kind arising from the sample code or your use thereof.

# Generate simulated data

In [ ]:
import os
import numpy as np
import seaborn as sns
import pandas as pd
from scipy.stats import bernoulli
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
    

f0 = np.random.normal(0, 1, 1000)
f1 = np.random.normal(0, 3, 1000)

distributions = [
    {"type": np.random.normal, "kwargs": {"loc": -2.5, "scale": 1}},
    {"type": np.random.normal, "kwargs": {"loc": 2.5, "scale": 1}},
]
coefficients = np.array([0.5, 0.5])
coefficients /= coefficients.sum()      # in case these did not add up to 1
sample_size = 1000

num_distr = len(distributions)
data = np.zeros((sample_size, num_distr))
for idx, distr in enumerate(distributions):
    data[:, idx] = distr["type"](size=(sample_size,), **distr["kwargs"])
random_idx = np.random.choice(np.arange(num_distr), size=(sample_size,), p=coefficients)
f2 = data[np.arange(sample_size), random_idx]

In [ ]:
p1 = sns.kdeplot(f0, shade=False, color="gray")
p1 = sns.kdeplot(f1, shade=False, color="blue")
p2 = sns.kdeplot(f2, shade=False, color="red")

## Generate plot

In [ ]:
data = {"Null":f0,
                  "Poorly-Separated":f1,
                  "Well-Separated":f2}
df = pd.DataFrame(data, columns = ['Null', 'Poorly-Separated','Well-Separated'])

fig, ax = plt.subplots(dpi=200)
use = ["Null","Poorly-Separated","Well-Separated"]
for u in use:
    sns.kdeplot(df[u], ax=ax, label=u)

plt.ylabel('$f_1(z)$', fontsize=16)
plt.xlabel('z', fontsize=16)

## Generate covariate functions

In [ ]:
### continous covariates
def generate_core_normal(n, m1, m2 ):
    mu1 = np.random.random(m1)

    cov1 = np.identity(m1) # independency

    mu2 = np.random.random(m2)
    cov2 = np.identity(m2)

    x1 = np.random.multivariate_normal(mu1, cov1, 1000)
    x2 = np.random.multivariate_normal(mu2, cov2, 1000)
    weight1 = np.random.normal(0, 4, m1)
    weight2 = np.random.lognormal(2,1, m2)

    h1_linear = x1 * weight1 
    h1_linear = h1_linear.sum(axis=1)
    h1 = (1/(1+np.exp(h1_linear))) 

    h2_linear = x2 * weight2
    h2_linear = h1_linear - h2_linear.sum(axis=1) 

    h2 = 1/(1+np.exp(h2_linear)) 
    
    return x1, x2, h1, h2

### unif covariates
def generate_core_unif(n, m1, m2):
    x1 = np.random.uniform(0, 1, size=(n, m1))
    x2 = np.random.uniform(0, 1, size=(n, m2))

    weight_a1 = np.random.lognormal(0, 1, m1)
    weight_b1 = np.random.lognormal(1, 1, m1)

    alpha1 =  (x1 * weight_a1).sum(axis=1)
    beta1 = (x1 * weight_b1).sum(axis=1)

    lr1 = LinearRegression()
    lr1.fit(x2, alpha1)
    alpha2 = lr1.predict(x2)

    lr2 = LinearRegression()
    lr2.fit(x2, beta1)
    beta2 = lr1.predict(x2)

    c1 = np.random.beta(alpha1, beta1)
    c2 =  np.random.beta(alpha2, beta2)
    
    return x1, x2, c1, c2

## Debug and check the generated covariates(linear)|

In [ ]:
### debug covariates are generated from multivariate normal with linear realtionship

m1 = 100
m2 = 5

mu1 = np.random.random(m1)

cov1 = np.identity(m1) # independency

mu2 = np.random.random(m2)
cov2 = np.identity(m2)

x1 = np.random.multivariate_normal(mu1, cov1, 1000)
x2 = np.random.multivariate_normal(mu2, cov2, 1000)
weight1 = np.random.normal(0, 1, m1)
weight2 = np.random.normal(2,1, m2)

h1_linear = x1 * weight1 
h1_linear = h1_linear.sum(axis=1)
h1 = (1/(1+np.exp(h1_linear))) 

h2_linear = x2 * weight2
h2_linear = h1_linear - h2_linear.sum(axis=1) 

h2 = 1/(1+np.exp(h2_linear)) 

## Debug and check the generated covariates(nonlinear)

In [ ]:
### debug covariates are generated from multivariate normal with non-linear realtionship

m1 = 100
m2 = 5

mu1 = np.random.random(m1)

cov1 = np.identity(m1) # independency

mu2 = np.random.random(m2)
cov2 = np.identity(m2)

x1 = np.random.multivariate_normal(mu1, cov1, 1000)
x2 = np.random.multivariate_normal(mu2, cov2, 1000)
weight1 = np.random.normal(0, 1, m1)
weight2 = np.random.normal(2,1, m2)

h1_linear = np.square(x1) * weight1
h1_linear = h1_linear.sum(axis=1)
h1 = (1/(1+np.exp(h1_linear))) 

h2_linear = np.square(x2) * weight2
h2_linear = h1_linear - h2_linear.sum(axis=1) 

h2 = 1/(1+np.exp(h2_linear)) 

## Debug and check the generated covariates(constant)

In [ ]:
m1 = 100
m2 = 5

mu1 = np.random.random(m1)

cov1 = np.identity(m1) # independency

mu2 = np.random.random(m2)
cov2 = np.identity(m2)

x1 = np.random.multivariate_normal(mu1, cov1, 1000)
x2 = np.random.multivariate_normal(mu2, cov2, 1000)

h2 = bernoulli.rvs(0.5, size=1000)

z_ws_continous = h2 * f2 + (1 - h2) * f0
z_ps_continous = h2 * f1 + (1 - h2) * f0

In [ ]:
np.sum(h1>0.5)

In [ ]:
h2[h2 > 0.5] = 1
h2[h2 <= 0.5] = 0

## Generate z

In [ ]:
# generate z
z_ws_continous = h2 * f2 + (1 - h2) * f0
z_ps_continous = h2 * f1 + (1 - h2) * f0

In [ ]:
# quick check h1 and h2
count = 0
for i in range(1000):
    x1, x2, h1, h2 = generate_core_unif(1000, 100, 10)
    
    if np.sum(h2>0.5) > np.sum(h1>0.5):
        count += 1
        
count

## Save data

In [ ]:
path = "data/simulation"
os.makedirs(path, exist_ok=True)

# save data
dat = pd.concat([pd.DataFrame(x1),pd.DataFrame(x2), pd.DataFrame(h2), pd.DataFrame(z_ws_continous), pd.DataFrame(z_ps_continous)], axis=1).to_numpy()
np.save(f"{path}/nonlinear_dat(n=1000,m=100).npy", dat)